In [ ]:
from IPython.display import HTML

HTML("""
<div style='font-size:15px; line-height:1.6; width:80%; margin:20px auto;'>
<p>
Over the past six months,  internal conflict across Sudan has intensified and dispersed, stretching from Khartoum and Gezira to Darfur, Kordofan, and the Red Sea region. The animated map illustrates how violence has shifted geographically, clustering around major population centers while also flaring in more remote areas.
</p>

<p>
Each red point represents an incident captured in the dataset, while the heat-map layer shows cumulative pressure points where conflict has been most sustained. The timeline slider allows viewers to trace how incidents build week by week, revealing patterns such as recurring clashes and emerging hotspots in central and western states.
</p>

<p>
Taken together, the visualization offers a dynamic view of how the conflict has evolved over half a year. It provides a foundation for further reporting into who is driving the violence, which communities are most affected, and how humanitarian conditions continue to deteriorate.
</p>
</div>
""")


In [10]:
#import and set up

import pandas as pd
import folium
from folium.plugins import HeatMap
folium.LayerControl()
from folium.plugins import MarkerCluster, HeatMap
from branca.element import Template, MacroElement
df = pd.read_csv("event_data_sdn.csv")

# load data - change form and get rid of na's then retrieve only the columns I'm looking to use and sort values by figure

df['figure'] = pd.to_numeric(df['figure'], errors='coerce')
df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')

conflict_df = df[df['displacement_type'].str.lower() == 'conflict']
conflict_df = conflict_df.dropna(subset=['figure', 'latitude', 'longitude', 'displacement_start_date', 'displacement_end_date'])

result = conflict_df.sort_values(       
    'figure', ascending=False    
)[['longitude', 'latitude', 'figure', 'locations_name', 'displacement_start_date', 'displacement_end_date']].reset_index(drop=True)

# base map - add all functions of data points (heat map layers, radius of points, timeline to m throughout)
m = folium.Map(
    location=[result['latitude'].mean(), result['longitude'].mean()],
    zoom_start=6
)

# Heatmap Layer - by location and size of displacement 

heat_data = result[['latitude', 'longitude', 'figure']].values.tolist()

HeatMap(
    heat_data,
    name="Heatmap (by figure)",
    radius=18,
    blur=12,
    max_zoom=10
).add_to(m)


# 3. Added circle radius (couldn't figure out how to do this in python, chat gpt suggested an html block)

from branca.element import Template, MacroElement

legend_html = """
<div style="
    position: fixed;
    bottom: 20px;
    left: 20px;
    width: 220px;
    height: 130px;
    background-color: white;
    border: 2px solid grey;
    z-index: 9999;
    font-size: 14px;
    padding: 10px;
    box-shadow: 3px 3px 5px rgba(0,0,0,0.3);
">
<b>Circle Size Legend</b><br>
<div style="margin-top: 5px;">
  <svg width="200" height="80">
    <circle cx="20" cy="20" r="5" fill="red" fill-opacity="0.6" stroke="red"></circle>
    <text x="40" y="24" font-size="12">Small displacement</text>

    <circle cx="20" cy="45" r="10" fill="red" fill-opacity="0.6" stroke="red"></circle>
    <text x="40" y="49" font-size="12">Medium displacement</text>

    <circle cx="20" cy="70" r="15" fill="red" fill-opacity="0.6" stroke="red"></circle>
    <text x="40" y="74" font-size="12">Large displacement</text>
  </svg>
</div>
</div>
"""

legend = MacroElement()
legend._template = Template(legend_html)
m.get_root().add_child(legend)

from folium.plugins import HeatMapWithTime

# Converted to datetime
conflict_df['date'] = pd.to_datetime(conflict_df['displacement_start_date'])

# Grouped strictly by day
time_groups = conflict_df.groupby(conflict_df['date'].dt.date)

# Constructed a list this can be used for the interactive timeline (showing not just shifting location but diplacement value as with heat map)
heatmap_time_data = [
    group[['latitude', 'longitude', 'figure']].values.tolist()
    for _, group in time_groups
]

# Cleaned index labels (YYYY-MM-DD) and add interactive timeline
index_labels = [str(date) for date in time_groups.groups.keys()]

HeatMapWithTime(
    heatmap_time_data,
    index=index_labels,
    radius=18,
    auto_play=True,
    max_opacity=0.8,
    name="Heatmap Over Time"
).add_to(m)


# Created formatted date string for timeline popups
conflict_df['date_str'] = conflict_df['date'].dt.strftime('%Y-%m-%d')

# Created a radius scaled to the displacement of figure
min_fig = conflict_df['figure'].min()
max_fig = conflict_df['figure'].max()

# Added a popup table (definitely got some guidance from chat gpt here)

def scale_radius(fig):
    # Scales values into a 4–20 px range
    return 4 + 16 * ((fig - min_fig) / (max_fig - min_fig))

conflict_df['radius'] = conflict_df['figure'].apply(scale_radius)

for _, row in conflict_df.iterrows():

    html_table = f"""
    <table style="width:220px; font-size:13px;">
        <tr><th colspan="2" style="text-align:center; background:#eee;">{row.get('locations_name', 'Unknown Location')}</th></tr>
        <tr><td><b>Figure</b></td><td>{int(row['figure'])}</td></tr>
        <tr><td><b>Start date</b></td><td>{row['displacement_start_date']}</td></tr>
        <tr><td><b>End date</b></td><td>{row.get('displacement_end_date', 'N/A')}</td></tr>
    </table>
    """

    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=row['radius'],
        color="red",
        fill=True,
        fill_color="red",
        fill_opacity=0.45,
        popup=folium.Popup(html_table, max_width=250)
    ).add_to(m)

#Create content of pop ups for displacement points (locations, size of displacement, time frame of displacement)
features = []
for _, row in conflict_df.iterrows():

    html_table = f"""
    <table style="width:220px; font-size:13px;">
        <tr><th colspan="2" style="text-align:center; background:#eee;">{row.get('locations_name', 'Unknown Location')}</th></tr>
        <tr><td><b>Figure</b></td><td>{int(row['figure'])}</td></tr>
        <tr><td><b>Start date</b></td><td>{row['displacement_start_date']}</td></tr>
        <tr><td><b>End date</b></td><td>{row.get('displacement_end_date', 'N/A')}</td></tr>
    </table>
    """

#create central circles for popups to exist within
    features.append({
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": [row['longitude'], row['latitude']]
        },
        "properties": {
            "time": row['date_str'],
            "popup": html_table,
            "style": {"color": "red", "fillColor": "red", "fillOpacity": 0.5},
            "iconstyle": {
                "radius": row['radius'],
                "color": "red",
                "fillColor": "red",
                "fillOpacity": 0.4
            }
        }
    })

m

